# Tensors

张量是一种特殊的数据结构，与数组和矩阵非常相似。
在 PyTorch 中，我们使用张量对模型的输入和输出以及模型的参数进行编码。
张量类似于 [NumPy的](https://numpy.org/) ndarrays，不同之处在于张量可以在 GPU 或其他硬件加速器上运行。 事实上，张量和 NumPy 数组通常可以共享相同的底层内存，从而无需复制数据（请参阅``bridge-to-np-labe``）。 张量也针对自动微分进行了优化（稍后我们将在 [Autograd](autogradqs_tutorial.html)_ 部分中看到更多相关信息）。 如果您熟悉 ndarrays，那么您将对 Tensor API 不陌生。 如果没有，请参照下面的文档！

In [ ]:
import torch
import numpy as np

## 初始化张量

张量可以通过多种方式初始化。 看看下面的例子： 

**直接从数据** 

可以直接从数据创建张量。 数据类型是自动推断的。


In [ ]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

**从NumPy数组**

可以从 NumPy 数组创建张量（反之亦然 - 请参阅``bridge-to-np-label``）。

In [ ]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

**从另一个张量：**

新张量保留参数张量的属性（形状、数据类型），除非明确覆盖。


In [ ]:
x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

**从随机值或常数值：**

``shape`` 是张量维度的元组。 在下面的函数中，它决定了输出张量的维数。


In [ ]:
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

--------------




## Tensor 的属性

Tensor 属性描述了它们的形状、数据类型和存储它们的设备。

In [ ]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

--------------




## 张量运算

[此处](https://pytorch.org/docs/stable/torch.html)全面描述了 100 多种张量运算，包括算术、线性代数、矩阵运算（转置、索引、切片）、采样等。

这些操作中的每一个都可以在 GPU 上运行（速度通常高于 CPU）。 如果您使用的是 Colab，请转到运行时 > 更改运行时类型 > GPU 来分配 GPU。

默认情况下，张量是在 CPU 上创建的。 我们需要使用 ``.to`` 方法（在检查 GPU 可用性之后）将张量显式移动到 GPU。 请记住，跨设备复制大型张量在时间和内存方面可能会很昂贵！



In [ ]:
# We move our tensor to the GPU if available
if torch.cuda.is_available():
    tensor = tensor.to("cuda")

尝试列表中的一些操作。

如果您熟悉 NumPy API，您会发现使用 Tensor API 轻而易举。

**标准的类似 numpy 的索引和切片：**

In [ ]:
tensor = torch.ones(4, 4)
print(f"First row: {tensor[0]}")
print(f"First column: {tensor[:, 0]}")
print(f"Last column: {tensor[..., -1]}")
tensor[:,1] = 0
print(tensor)

**连接张量** 您可以使用 ``torch.cat`` 沿给定维度连接一系列张量。另请参阅 [torch.stack](https://pytorch.org/docs/stable/generated/torch.stack.html)，这是另一个与 ``torch.cat`` 略有不同的张量连接选项。

In [ ]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

**算术运算**

In [ ]:
# This computes the matrix multiplication between two tensors. y1, y2, y3 will have the same value
# ``tensor.T`` returns the transpose of a tensor
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(y1)
torch.matmul(tensor, tensor.T, out=y3)


# This computes the element-wise product. z1, z2, z3 will have the same value
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)

**单元素张量** 如果你有一个单元素张量，例如通过将一个张量的所有值聚合成一个值，你可以使用 ``item()`` 将它转换为 Python 数值：

In [ ]:
agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg_item))

**就地操作** 将结果存储到操作数中的操作称为就地操作。 它们由 ``_`` 后缀表示。例如：``x.copy_(y)``、``x.t_()``，将改变``x``。

In [ ]:
print(f"{tensor} \n")
tensor.add_(5)
print(tensor)

<div class="alert alert-info"><h4>注意</h4><p>就地操作可以节省一些内存，但在计算导数时可能会出现问题，因为会立即丢失历史记录。 因此，不鼓励使用它们。</p></div>

--------------




## 与 NumPy 桥接

CPU 上的 Tensor 和 NumPy 数组可以共享它们的底层内存位置，改变一个就会改变另一个。

### Tensor to NumPy array



In [ ]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

A change in the tensor reflects in the NumPy array.



In [ ]:
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

### NumPy array to Tensor



In [ ]:
n = np.ones(5)
t = torch.from_numpy(n)

Changes in the NumPy array reflects in the tensor.



In [ ]:
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")